In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from collections import defaultdict
from pathlib import Path
from natsort import natsort_keygen, natsorted # to naturally sort string columns

plt.style.use('matplotlibrc')

KB = 1024
MB = 1024 * 1024
GB = 1024 * 1024 * 1024

In [ ]:
def set_size(fraction_width=0.95, fraction_height=0.25):
    width_pt = 241.14749 # column width in pt
    height_pt = 626.0 # page height in pt

    fig_width_pt = width_pt * fraction_width
    fig_height_pt = height_pt * fraction_height
    inches_per_pt = 1 / 72.27

    fig_width_in = fig_width_pt * inches_per_pt
    fig_height_in = fig_height_pt * inches_per_pt

    return (fig_width_in, fig_height_in)

In [ ]:
# define color to use throughout the notebook
paired = matplotlib.colormaps['Paired']
tab10 = matplotlib.colormaps['tab20']
dark2 = matplotlib.colormaps['Dark2']

c_st =  tab10(0)
c_decomp = tab10(1)
c_rdb_w_pj = tab10(2)
c_rdb_wo_pj = tab10(4)
c_redundancy = tab10(14)

c_rm1 = dark2(0)
c_rm2 = dark2(2)
c_rm3 = dark2(5)
c_rm4 = dark2(3)

colormap = {
    'Single Table': c_st,
    'Decompose': c_decomp,
    'RDB w/ post-join info': c_rdb_w_pj,
    'Result DB': c_rdb_w_pj,
    'RDB w/o post-join info': c_rdb_wo_pj,
    '0. Single Table': c_st,
    'RM1. Dynamic SELECT DISTINCT': c_rm1,
    'RM2. Materialized SELECT DISTINCT': c_rm2,
    'RM3. Dynamic Subquery': c_rm3,
    'RM4. Materialized Subquery': c_rm4,
    'Redundancy': c_redundancy
}

# Result Set Sizes

## Join-Order Benchmark

In [ ]:
def compression_ratio(uncompressed, compressed):
    """
    Compute the compression ratio.

    Returns:
    float: Compression ratio.
    """
    # assert uncompressed != 0, "uncompressed must not be zero"
    if compressed == 0:
        return 0
    return round(uncompressed / compressed, 2)

data = pd.read_csv('../result-set-sizes/result-set-sizes.csv')
data = data.drop(['relation', 'count'], axis=1)
data = data.groupby(by=['database', 'query', 'method'], as_index=False).sum()
data = data.sort_values(by=['query', 'method'], key=natsort_keygen())
data['size'] = round(data['size'] / KB, 2)

single_table = data[data['method'] == 'Single Table']
rdb_w_post_join_info = data[data['method'] == 'rdb_w_post_join_info']
rdb_w_post_join_info['compression_ratio'] = [ compression_ratio(baseline, new) for baseline, new in zip(single_table['size'], rdb_w_post_join_info['size']) ]
rdb_wo_post_join_info = data[data['method'] == 'rdb_wo_post_join_info']
rdb_wo_post_join_info['compression_ratio'] = [ compression_ratio(baseline, new) for baseline, new in zip(single_table['size'], rdb_wo_post_join_info['size']) ]

display(single_table)
display(rdb_w_post_join_info)
display(rdb_wo_post_join_info)

## Synthetic

Star-schema

In [ ]:
num_dim_tables = 4 # number of dimension tables
dim_payload = 20 # byte
dim_row_size = 4 + dim_payload # 4 byte primary key + payload

fact_payload = 20
fact_row_size = fact_payload + num_dim_tables * 4 # 4 byte foreign key per dimension table + payload

dim1_size = 60
dim2_size = 60
dim3_size = 60
dim4_size = 60
fact_size = dim1_size * dim2_size * dim3_size * dim4_size

fact_size_selectivities = np.arange(0.1, 1.1, 0.1)
fact_sizes = [ fact_size * sel for sel in fact_size_selectivities ]

single_table = [ ((fact_row_size + dim_row_size * num_dim_tables) * f_size) / (1024**2) for f_size in fact_sizes ]
rdb_with_post_join = [ (dim_row_size * (dim1_size + dim2_size + dim3_size + dim4_size) + fact_row_size * f_size) / (1024**2) for f_size in fact_sizes ]
rdb_without_post_join = [ (dim_row_size * (dim1_size + dim2_size + dim3_size + dim4_size)) / (1024**2) for _ in fact_sizes ]


fig, ax = plt.subplots(layout='constrained', figsize=(4,3))

plt.plot(fact_sizes, single_table, 'o--', color=colormap['Single Table'], label='Single Table')
plt.plot(fact_sizes, rdb_with_post_join, 'x--', color=colormap['RDB w/ post-join info'], label='RDB w/ post-join info')
plt.plot(fact_sizes, rdb_without_post_join, '^--', color=colormap['RDB w/o post-join info'], label='RDB w/o post-join info')


ax.set_xlabel('\#tuples in Fact table')
ax.set_ylabel('Size [MiB]')
ax.set_title('Synthetic Result Set Sizes')
# ax.set_xticks(x + width, queries)
ax.legend(loc='upper left', ncols=1)
# ax.set_ylim(0)
# ax.set_yscale('log')

fig.savefig('synthetic-result-sizes.pdf')

In [ ]:
num_dim_tables = 4 # number of dimension tables
dim_payload = 20 # byte
dim_row_size = 4 + dim_payload # 4 byte primary key + payload
dim_table_size = 60
selectivities = np.arange(0.1, 1.1, 0.1)
dim_table_sizes = [ int(dim_table_size * sel) for sel in selectivities ]

fact_payload = 20
fact_row_size = 4 + num_dim_tables * 4 + fact_payload # 4 byte pk + 4 byte foreign key per dimension table + payload
fact_size = dim_table_size**num_dim_tables
fact_table_sizes = [ dim_size**num_dim_tables for dim_size in dim_table_sizes ]

single_table = [ ((fact_row_size + dim_row_size * num_dim_tables) * f_size) / MB for f_size in fact_table_sizes ]
rdb_with_post_join = ([ ((f_size * fact_row_size) + (dim_row_size * d_size * num_dim_tables)) / MB
                       for f_size, d_size in zip(fact_table_sizes, dim_table_sizes) ])
rdb_without_post_join = [ (dim_row_size * d_size * num_dim_tables) / MB for d_size in dim_table_sizes ]

### Plot
fig, ax = plt.subplots(layout='constrained', figsize=(3,2))
# display(*single_table/rdb_with_post_join)
# for s, r in zip(single_table, rdb_with_post_join):
#     display(s/r)
# display(rdb_with_post_join)
plt.plot(selectivities, single_table, 'o--', color=colormap['Single Table'], label='Single Table')
plt.plot(selectivities, rdb_with_post_join, 'x--', color=colormap['RDB w/ post-join info'], label='RDB w/ post-join info')
plt.plot(selectivities, rdb_without_post_join, '^--', color=colormap['RDB w/o post-join info'], label='RDB w/o post-join info')

ax.set_xlabel('Dimension table selectivities | redundancy')
ax.set_ylabel('Size [MiB]')
# ax.set_title('Synthetic Result Set Sizes')
# ax.set_xticks(selectivities)
ax.legend(loc='upper left', ncols=1)
# ax.set_ylim(0)
# ax.set_yscale('log')

fig.savefig('synthetic-result-sizes.pdf')

# TODO: make a plot that varies (x-axis) the redundancy or shows (y-axis) the redundancy for different selectivities

In [ ]:
num_dim_tables = 4 # number of dimension tables
dim_payload = 20 # byte
dim_row_size = 4 + dim_payload # 4 byte primary key + payload
dim_table_size = 60
selectivities = np.arange(0.1, 1.1, 0.1)
dim_table_sizes = [ int(dim_table_size * sel) for sel in selectivities ]

fact_payload = 20
fact_row_size = 4 + num_dim_tables * 4 + fact_payload # 4 byte pk + 4 byte foreign key per dimension table + payload
fact_size = dim_table_size**num_dim_tables
fact_table_sizes = [ dim_size**num_dim_tables for dim_size in dim_table_sizes ]

single_table = [ ((fact_row_size + dim_row_size * num_dim_tables) * f_size) / MB for f_size in fact_table_sizes ]
rdb_with_post_join = ([ ((f_size * fact_row_size) + (dim_row_size * d_size * num_dim_tables)) / MB
                       for f_size, d_size in zip(fact_table_sizes, dim_table_sizes) ])
rdb_without_post_join = [ (dim_payload * d_size * num_dim_tables + f_size * (fact_payload)) / MB for f_size, d_size in zip(fact_table_sizes, dim_table_sizes) ]

print(single_table)
print(rdb_with_post_join)
### Plot
fig, ax = plt.subplots(figsize=set_size(fraction_width=1.0, fraction_height=0.22), layout='constrained')
# display(*single_table/rdb_with_post_join)
# for s, r in zip(single_table, rdb_with_post_join):
#     display(s/r)
display(rdb_with_post_join)
ax.plot(selectivities, single_table, 'o--', color=colormap['Single Table'], label='Single Table')
ax.plot(selectivities, rdb_with_post_join, 'x--', color=colormap['RDB w/ post-join info'], label='RDB w/ post-join info')
ax.plot(selectivities, rdb_without_post_join, '^--', color=colormap['RDB w/o post-join info'], label='RDB w/o post-join info')

ax.fill_between(selectivities, single_table, rdb_with_post_join, color=colormap['Redundancy'], label='Redundancy')

ax.set_xlabel('Selectivity')
ax.set_ylabel('Result Set Size [MiB]')
# ax.set_title('Synthetic Result Set Sizes')
# ax.set_xticks(selectivities)
# fig.legend(ncols=int(len(method_times)/1), bbox_to_anchor=(0.5, 1), loc='lower center')
ax.legend(loc='upper left', ncols=1)
# ax.set_ylim(0)
# ax.set_yscale('log')

fig.savefig('synthetic-result-sizes.pdf', bbox_inches='tight')

# TODO: make a plot that varies (x-axis) the redundancy or shows (y-axis) the redundancy for different selectivities
# showing the redundancy does not really work well because both RDB approaches do not incur any redundancy

In [ ]:
num_dim_tables = 4 # number of dimension tables
dim_table_size = 60

dim_payloads = [0, 50, 100, 150, 200]

fact_payload = 20
fact_row_size = 4 + fact_payload + num_dim_tables * 4 # 4 byte pk + payload + 4 byte foreign key per dimension table
fact_size = dim_table_size**num_dim_tables

single_table = []
rdb_with_post_join = []
rdb_without_post_join = []

for dim_payload in dim_payloads:
    dim_row_size = 4 + dim_payload
    single_table_row_size = fact_row_size + num_dim_tables * dim_row_size
    single_table.append((single_table_row_size * fact_size) / MB)
    rdb_with_post_join.append((fact_row_size * fact_size + (num_dim_tables * dim_row_size * dim_table_size)) / MB)
    rdb_without_post_join.append((num_dim_tables * dim_row_size * dim_table_size) / MB)

# single_table = [ ((fact_row_size + (4 + dim_payload) * dim_table_size * num_dim_tables) * fact_size) / MB for dim_payload in dim_payloads ]
# rdb_with_post_join = ([ (fact_size * fact_row_size + (4 + dim_payload) * dim_table_size * num_dim_tables) / MB for dim_payload in dim_payloads ])
# rdb_without_post_join = [ ((4 + dim_payload) * dim_table_size * num_dim_tables) / MB for dim_payload in dim_payloads ]

# display(single_table)
# display(rdb_with_post_join)
### Plot
fig, ax = plt.subplots(layout='constrained', figsize=(3,2))
# display(*single_table/rdb_with_post_join)
for s, r in zip(single_table, rdb_with_post_join):
    display(s/r)

# display(rdb_with_post_join)
plt.plot(dim_payloads, single_table, 'o--', color=colormap['Single Table'], label='Single Table')
plt.plot(dim_payloads, rdb_with_post_join, 'x--', color=colormap['RDB w/ post-join info'], label='RDB w/ post-join info')
plt.plot(dim_payloads, rdb_without_post_join, '^--', color=colormap['RDB w/o post-join info'], label='RDB w/o post-join info')

ax.set_xlabel('payload of each dimension table [bytes]')
ax.set_ylabel('Size [MiB]')
# ax.set_title('Synthetic Result Set Sizes')
# ax.set_xticks(x + width, queries)
ax.legend(loc='upper left', ncols=1)
# ax.set_ylim(0)
# ax.set_yscale('log')

fig.savefig('synthetic-result-sizes-payload.pdf')

#### Memory Consumption

## Join-Order Benchmark

In [ ]:
data = pd.read_csv('./memory-consumption.csv')
data = data.sort_values(by=['benchmark', 'query'], key=natsort_keygen())
queries = data['query'].unique()
method_sizes = {
    "Single Table": [],
    "Result DB": []
}


for index, row in data.iterrows():
    benchmark = row['benchmark']
    query = row['query']
    resultdb = row['resultdb']
    size = row['size_MiB']
    if (resultdb == 0):
        method_sizes['Single Table'].append(size)
    else:
        method_sizes['Result DB'].append(size)

x = np.arange(len(queries))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained', figsize=(4,3))
for m, size in method_sizes.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, size, width, color=colormap[m], label=m)
    # ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Size [MiB] (log-scale)')
ax.set_title('JOB Memory Consumption')
ax.set_xticks(x + width/2, queries)
ax.legend(loc='upper right', ncols=1)
# ax.set_ylim(0)
ax.set_yscale('log')

fig.savefig('job-memory-consumption.pdf')

# Rewrite Methods

## Join-Order Benchmark

In [ ]:
def preprocess_data(data):
    # construct the following dictionary:
    # {
    #   'RM0': [q0_time, q1_time, ..., qn-1_time]
    #   'RM1': [q0_time, q1_time, ..., qn-1_time]
    # }
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    method_times = defaultdict(list)
    for _, row in data.iterrows():
        method_times[row['method']].append(row['time'])
    return method_times

def bar_plot(data, data_transfer, filename):
    queries = natsorted(data['query'].unique())
    method_times = preprocess_data(data[data['data_transfer'] == data_transfer])
    x = np.arange(len(queries))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0
    fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.26), layout='constrained')
    for method, times in method_times.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, times, width, color=colormap[method], label=method)
        # ax.bar_label(rects, padding=3, size=8)
        multiplier += 1
    ax.set_xticks(x + width + width/2, queries)
    ax.set_xlabel('JOB Queries')
    ax.legend(loc='upper left')
    # fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')
    ax.xaxis.grid(False) # Disable grid lines on the x-axis
    
    log_scale = True
    if log_scale:
        ax.set_ylabel('Query Execution Time [ms] (log-scale)')
        ax.set_yscale('log')
    else:
        ax.set_ylabel('Query Execution Time [ms]')
        ax.set_ylim(0)
        
    fig.savefig(filename, bbox_inches='tight')

data = pd.read_csv('../rewrite-methods/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
exclude_queries = [
    # 'q13a',
    # 'q17a'
]
data = data[~data['query'].isin(exclude_queries)]
exclude_methods = [
    '0. Single Table'
]
data = data[~data['method'].isin(exclude_methods)]

bar_plot(data, False, 'job-rewrite-methods.pdf')

In [ ]:
def overhead(baseline, new):
    assert baseline != 0, "baseline must not be zero"
    if (new <= baseline): # new is faster -> improvement
        return -((baseline - new) / baseline) * 100
    else: # new is slower -> overhead 
        return ((new - baseline) / baseline) * 100
        
data = pd.read_csv('../rewrite-methods/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
data = data[data['data_transfer'] == False]
exclude_queries = [
    # 'q13a',
]
data = data[~data['query'].isin(exclude_queries)]
# methods = data['method'].unique()[1:]

single_table = data[data['method'] == '0. Single Table']
rewrite_methods = data[data['method'] != '0. Single Table']

# NOTE: be careful as ties are included (handle them manually)
min_values = rewrite_methods.groupby(by=['query'])['time'].transform('min')
best_rewrite_method = rewrite_methods[rewrite_methods['time'] == min_values]

# ensure that both dataframes are sorted correctly
single_table = single_table.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
best_rewrite_method = best_rewrite_method.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly

display(single_table)
display(best_rewrite_method)

assert len(single_table) == len(best_rewrite_method)

for (index1, st), (index2, rm) in zip(single_table.iterrows(), best_rewrite_method.iterrows()):
    print(f"{st['query']}: {round(overhead(int(st['time']), int(rm['time'])), 1)}")
    # print(f"{st['query']}: {round(int(st['time']) / int(rm['time']), 2)}")

In [ ]:
def compute_improvement_factor(baseline, new):
    """
    Compute the improvement factor of a new method against a baseline.

    Args:
    baseline (float): Performance metric of the baseline method.
    new (float): Performance metric of the new method.

    Returns:
    float: Improvement factor as a percentage.
    """
    assert baseline != 0, "baseline must not be zero"
    # return ((baseline - new) / baseline) * 100
    if (new <= baseline):
        return ((baseline / new) - 1)
    else:
        return (-(new / baseline) + 1)

def bar_plot(data, query, ax, col):
    data = data[data['query'] == query]
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    methods = data['method'].values[1:] # drop the first method which is the single-table
    times = data['time'].values
    improvement_factors = [ compute_improvement_factor(times[0], t) for t in times[1:] ]
    x = np.arange(len(methods[1:]))  # the label locations
    min_ylim = -1
    max_ylim = 1
    ax.set_ylim(min_ylim, max_ylim)
    width = 0.01  # the width of the bars
    multiplier = 0
    for method, improv_factor in zip(methods, improvement_factors):
        offset = width * multiplier
        bars = ax.bar(offset, improv_factor, width, color=colormap[method], label=method)
        for bar in bars:
            height = bar.get_height()
            if height < min_ylim:
                ax.text(bar.get_x() + bar.get_width() / 2, 0, f'{round(height, 2)}', ha='center', va='bottom', fontsize=8)
            elif height > max_ylim:
                ax.text(bar.get_x() + bar.get_width() / 2, -0.2, f'{round(height, 2)}', ha='center', va='bottom', fontsize=8)
        multiplier += 1
    ax.set_xticks([])
    if col != 0:
        pass
    else:
        ax.set_ylabel("Improvement factor")
        # ax.set_yticks([-100, -50, 0, 50, 100])

    ax.set_xlabel(query)
    ax.axhline(0, color='black', linewidth=0.8)  # Add a horizontal line at y=0

data = pd.read_csv('../rewrite-methods/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
data = data[data['data_transfer'] == True]
methods = data['method'].unique()[1:]

exclude_queries = [
    'q13a',
]
data = data[~data['query'].isin(exclude_queries)]

queries = natsorted(data['query'].unique())

# create plot
rows = 1
cols = len(queries)
fig, axes = plt.subplots(rows, cols, sharey=True, figsize=(10, 1.5))

cur_row = 0
cur_col = 0
for q in queries:
    if (rows == 1):
        bar_plot(data, q, axes[cur_col], cur_col)
    else:
        bar_plot(data, q, axes[cur_row, cur_col], cur_col)
    cur_col = 0 if cur_col == (cols - 1) else cur_col + 1
    cur_row = cur_row + 1 if cur_col == 0 else cur_row
## remove all subplots that do not contain data (in case we have an "uncomplete" row)
for ax in axes.flat: # access each axes object via axs.flat
    ## check if something was plotted 
    if not bool(ax.has_data()):
        fig.delaxes(ax) ## delete if nothing is plotted in the axes obj

# legend
if (rows == 1):
    handles, labels = axes[0].get_legend_handles_labels()
else:
    handles, labels = axes[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncols=len(methods), bbox_to_anchor=(0.5, 1.1))

fig.savefig('job-rewrite-methods-improvement_factor.pdf', bbox_inches='tight')
plt.show()

## Synthetic

### Varying number of joins

In [ ]:
def preprocess_data(data):
    # construct the following dictionary:
    # {
    #   'RM0': [q0_time, q1_time, ..., qn-1_time]
    #   'RM1': [q0_time, q1_time, ..., qn-1_time]
    # }
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    method_times = defaultdict(list)
    for _, row in data.iterrows():
        method_times[row['method']].append(row['time'])
    return method_times

def bar_plot(data, data_transfer, filename):
    queries = natsorted(data['query'].unique())
    method_times = preprocess_data(data[data['data_transfer'] == data_transfer])
    x = np.arange(len(queries))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0
    fig, ax = plt.subplots(layout='constrained', figsize=(4,3))
    for method, times in method_times.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, times, width, color=colormap[method], label=method)
        # ax.bar_label(rects, padding=3, size=8)
        multiplier += 1

    ax.set_ylabel('Query Execution Time [ms] (log-scale)')
    title = f'JOB Rewrite Method Runtime w/{"o" if not data_transfer else ""} data transfer in PostgreSQL'
    # ax.set_title(title)
    number_of_joins = [ q.rsplit('_', 1)[1] for q in queries ]
    ax.set_xticks(x + width + width/2, number_of_joins)
    ax.set_xlabel('number of joins')
    # ax.legend()
    # ax.set_ylim(0)
    ax.set_yscale('log')
    # fig.legend()
    fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')
    
    fig.savefig(filename)

data = pd.read_csv('../star-schema/rewrite-results.csv')
data = data[data['query'].str.contains('star_joins_')]
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)

method_times = preprocess_data(data)
single_table = method_times['0. Single Table']
rdb_with_post_join = method_times['RM1. Dynamic SELECT DISTINCT']

# for s, r in zip(single_table, rdb_with_post_join):
#     display(compute_improvement_factor(s,r))

exclude_methods = [
    '0. Single Table'
]
data = data[~data['method'].isin(exclude_methods)]

bar_plot(data, True, 'synthetic-rewrite-methods-joins.pdf')
# bar_plot(data, False, 'job-rewrite-methods_no-data-transfer.pdf')

### Varying number of projections

In [ ]:
query_names = {
    '2_dim': '2x dim',
    '3_dim': '3x dim',
    '4_dim': '4x dim',
    'fact_1_dim': 'fact + 1x dim',
    'fact_2_dim': 'fact + 2x dim',
    'fact_3_dim': 'fact + 3x dim',
    'fact_4_dim': 'fact + 4x dim'
}

def preprocess_data(data):
    # construct the following dictionary:
    # {
    #   'RM0': [q0_time, q1_time, ..., qn-1_time]
    #   'RM1': [q0_time, q1_time, ..., qn-1_time]
    # }
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    method_times = defaultdict(list)
    for _, row in data.iterrows():
        method_times[row['method']].append(row['time'])
    return method_times

def bar_plot(data, data_transfer, filename):
    queries = natsorted(data['query'].unique())
    method_times = preprocess_data(data[data['data_transfer'] == data_transfer])
    x = np.arange(len(queries))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0
    fig, ax = plt.subplots(layout='constrained', figsize=(4,3))
    for method, times in method_times.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, times, width, color=colormap[method], label=method)
        # ax.bar_label(rects, padding=3, size=8)
        multiplier += 1

    ax.set_ylabel('Query Execution Time [ms] (log-scale)')
    title = f'JOB Rewrite Method Runtime w/{"o" if not data_transfer else ""} data transfer in PostgreSQL'
    # ax.set_title(title)
    names = [ query_names[q.split('_', 2)[2]] for q in queries ]
    ax.set_xticks(x + width + width/2, names, rotation=45)
    ax.set_xlabel('projected attributes')
    # ax.legend()
    # ax.set_ylim(0)
    ax.set_yscale('log')
    # fig.legend()
    fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')
    
    fig.savefig(filename)

data = pd.read_csv('../star-schema/rewrite-results.csv')
data = data[data['query'].str.contains('star_proj_')]
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)

exclude_methods = [
    '0. Single Table'
]
data = data[~data['method'].isin(exclude_methods)]

bar_plot(data, True, 'synthetic-rewrite-methods-projections.pdf')
# bar_plot(data, False, 'job-rewrite-methods_no-data-transfer.pdf')

### Varying selectivity

In [ ]:
def preprocess_data(data):
    # construct the following dictionary:
    # {
    #   'RM0': [q0_time, q1_time, ..., qn-1_time]
    #   'RM1': [q0_time, q1_time, ..., qn-1_time]
    # }
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    method_times = defaultdict(list)
    for _, row in data.iterrows():
        method_times[row['method']].append(row['time'])
    return method_times

def bar_plot(data, data_transfer, filename):
    queries = natsorted(data['query'].unique())
    selectivities = [ int(q.rsplit('_', 1)[1]) / 100 for q in data['query'].unique() ]
    method_times = preprocess_data(data[data['data_transfer'] == data_transfer])
    x = np.arange(len(selectivities))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0
    fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.26), layout='constrained')
    for method, times in method_times.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, times, width, color=colormap[method], label=method)
        # ax.bar_label(rects, padding=3, size=8)
        multiplier += 1
    selectivities = [ int(q.rsplit('_', 1)[1]) / 100 for q in queries ]
    ax.set_xticks(x + width + width/2, selectivities)
    ax.set_xlabel('Selectivity')
    ax.xaxis.grid(False) # Disable grid lines on the x-axis
    ax.legend()

    log_scale = True
    if log_scale:
        ax.set_ylabel('Query Execution Time [ms] (log-scale)')
        ax.set_yscale('log')
    else:
        ax.set_ylabel('Query Execution Time [ms]')
        ax.set_ylim(0)
    
    fig.savefig(filename, bbox_inches='tight')

data = pd.read_csv('../star-schema/rewrite-results.csv')
data = data[data['query'].str.contains('star_sel_')]
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)

exclude_methods = [
    '0. Single Table'
]
data = data[~data['method'].isin(exclude_methods)]

# bar_plot(data, True, 'synthetic-rewrite-methods-selectivity.pdf')
bar_plot(data, False, 'synthetic-rewrite-methods-selectivity.pdf')

In [ ]:
# def preprocess_data(data):
#     # construct the following dictionary:
#     # {
#     #   'RM0': [q0_time, q1_time, ..., qn-1_time]
#     #   'RM1': [q0_time, q1_time, ..., qn-1_time]
#     # }
#     data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
#     method_times = defaultdict(list)
#     for _, row in data.iterrows():
#         method_times[row['method']].append(row['time'])
#     return method_times

# data = pd.read_csv('../star-schema/rewrite-results.csv')
# data = data[data['query'].str.contains('star_sel_')]
# data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
# data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
# data = data.drop(['database', 'system'], axis=1)
# exclude_methods = [
#     '0. Single Table'
# ]
# data = data[~data['method'].isin(exclude_methods)]
# data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly

# selectivities = [ int(q.rsplit('_', 1)[1]) / 100 for q in data['query'].unique() ]

# method_times = preprocess_data(data)
# fig, ax = plt.subplots(layout='constrained', figsize=(4,3))
# for method, times in method_times.items():
#     ax.plot(selectivities, times, color=colormap[method], label=method)


# ax.set_ylabel('Query Execution Time [ms]')
# ax.set_xlabel('selectivity on all dimension tables')
# ax.legend()
# # ax.set_ylim(0)
# # ax.set_yscale('log')
# # fig.legend()
# # fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

# fig.savefig('synthetic-rewrite-methods-selectivities.pdf', bbox_inches='tight')
# plt.show()

In [ ]:
def overhead(baseline, new):
    assert baseline != 0, "baseline must not be zero"
    if (new <= baseline): # new is faster -> improvement
        return -((baseline - new) / baseline) * 100
    else: # new is slower -> overhead 
        return ((new - baseline) / baseline) * 100
        
data = pd.read_csv('../star-schema/rewrite-results.csv')
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
data = data[data['query'].str.contains('star_sel_')]
data = data[data['data_transfer'] == True]

single_table = data[data['method'] == '0. Single Table']
rewrite_methods = data[data['method'] != '0. Single Table']

# NOTE: be careful as ties are included (handle them manually)
min_values = rewrite_methods.groupby(by=['query'])['time'].transform('min')
best_rewrite_method = rewrite_methods[rewrite_methods['time'] == min_values]

# ensure that both dataframes are sorted correctly
single_table = single_table.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
best_rewrite_method = best_rewrite_method.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly

display(single_table)
display(best_rewrite_method)

assert len(single_table) == len(best_rewrite_method)

for (index1, st), (index2, rm) in zip(single_table.iterrows(), best_rewrite_method.iterrows()):
    print(f"{st['query']}: {round(overhead(int(st['time']), int(rm['time'])), 1)}")
    # print(f"{st['query']}: {round(int(st['time']) / int(rm['time']), 2)}")

In [ ]:
def compute_improvement_factor(baseline, new):
    """
    Compute the improvement factor of a new method against a baseline.

    Args:
    baseline (float): Performance metric of the baseline method.
    new (float): Performance metric of the new method.

    Returns:
    float: Improvement factor as a percentage.
    """
    assert baseline != 0, "baseline must not be zero"
    # return ((baseline - new) / baseline) * 100
    if (new <= baseline):
        return ((baseline / new) - 1)
    else:
        return (-(new / baseline) + 1)

def bar_plot(data, query, ax, col):
    data = data[data['query'] == query]
    data = data.sort_values(by=['query', 'method'], key=natsort_keygen()) # ensure that data is sorted correctly
    methods = data['method'].values[1:] # drop the first method which is the single-table
    times = data['time'].values
    improvement_factors = [ compute_improvement_factor(times[0], t) for t in times[1:] ]
    x = np.arange(len(methods[1:]))  # the label locations
    min_ylim = -7
    max_ylim = 7
    ax.set_ylim(min_ylim, max_ylim)
    width = 0.01  # the width of the bars
    multiplier = 0
    for method, improv_factor in zip(methods, improvement_factors):
        offset = width * multiplier
        bars = ax.bar(offset, improv_factor, width, color=colormap[method], label=method)
        for bar in bars:
            height = bar.get_height()
            if height < min_ylim:
                ax.text(bar.get_x() + bar.get_width() / 2, 0, f'{round(height, 1)}', ha='center', va='bottom', fontsize=8)
            elif height > max_ylim:
                ax.text(bar.get_x() + bar.get_width() / 2, -0.2, f'{round(height, 1)}', ha='center', va='bottom', fontsize=8)
        multiplier += 1
    ax.set_xticks([])
    if col != 0:
        pass
    else:
        ax.set_ylabel("Improvement factor (\%)")
        # ax.set_yticks([-800, -400, 0, 400, 800])

    ax.axhline(0, color='black', linewidth=0.8)  # Add a horizontal line at y=0
    ax.set_xlabel(f"{int(query.rsplit('_', 1)[1]) / 100}")


data = pd.read_csv('../star-schema/rewrite-results.csv')
data = data[data['query'].str.contains('star_sel_')]
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer', 'run'], as_index=False).sum()
data = data.groupby(by=['database', 'system', 'query', 'method', 'data_transfer'], as_index=False).median().drop(['run', 'num_query_internal'], axis=1)
data = data.drop(['database', 'system'], axis=1)
data = data[data['data_transfer'] == True]
methods = data['method'].unique()[1:]
queries = natsorted(data['query'].unique())

# create plot
rows = 1
cols = len(queries)
fig, axes = plt.subplots(rows, cols, sharey=True, figsize=(10, 1.5))

cur_row = 0
cur_col = 0
for q in queries:
    if (rows == 1):
        bar_plot(data, q, axes[cur_col], cur_col)
    else:
        bar_plot(data, q, axes[cur_row, cur_col], cur_col)
    cur_col = 0 if cur_col == (cols - 1) else cur_col + 1
    cur_row = cur_row + 1 if cur_col == 0 else cur_row
## remove all subplots that do not contain data (in case we have an "uncomplete" row)
for ax in axes.flat: # access each axes object via axs.flat
    ## check if something was plotted 
    if not bool(ax.has_data()):
        fig.delaxes(ax) ## delete if nothing is plotted in the axes obj

# legend
if (rows == 1):
    handles, labels = axes[0].get_legend_handles_labels()
else:
    handles, labels = axes[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncols=len(methods), bbox_to_anchor=(0.5, 1.1))
# Add a common x-axis label for all subplots
fig.text(0.5, -0.1, 'Selectivity', ha='center', va='center')

fig.savefig('synthetic-rewrite-methods-improvement_factor.pdf', bbox_inches='tight')
plt.show()

# RESULTDB Algorithm

## Join-Order Benchmark

In [ ]:
# construct the following dictionary:
# {
#   'Single-Table': [q4_time, q5_time, ...]
#   'Single-Table + Denom': [q4_time, q5_time, ...]
#   'Result-DB': [q4_time, q5_time, ...]
# }
algorithm_times = {
    'Single-Table': [],
    'Single-Table + Decompose': [],
    'Result-DB': []
}
# ensure that `queries` is sorted!
queries = [
    'q1a',
    'q3b',
    'q4a',
    'q5b',
    'q7b'
]
processed_queries = []
num_queries = 0
for query in queries:
    result_file = Path(f"../{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query, f'experiment query: {experiment[0]} does not match query {query} of current file'

    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            algorithm_times['Single-Table'].append(execution_time)
        elif "decompose" in algorithm:
            algorithm_times['Single-Table + Decompose'].append(execution_time)
        elif "resultdb" in algorithm:
            algorithm_times['Result-DB'].append(execution_time)
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"
# display(algorithm_times)
            
# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0
fig, ax = plt.subplots(layout='constrained', figsize=(4,3))
for algorithm, time in algorithm_times.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, time, width, label=algorithm)
    # ax.bar_label(rects, padding=3, size=8)
    multiplier += 1

ax.set_ylabel('Runtime [ms]')
ax.set_title('JOB Algorithm Runtime in mutable')
ax.set_xticks(x + width, processed_queries)
ax.legend()
ax.set_ylim(0)
# ax.set_yscale('log')
# fig.legend()
# fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

fig.savefig('job-benchmarks.pdf')

In [ ]:
# construct the following dictionary:
# {
#   'Single-Table': [q4_time, q5_time, ...]
#   'Single-Table + Decompose': [q4_time, q5_time, ...]
#   'Result-DB': [q4_time, q5_time, ...]
# }
# ensure that `queries` is sorted!
queries = [
    'q1a',
    'q3b',
    'q4a',
    'q5b',
    # 'q7b'
]

single_table_decompose = []
resultdb = []
processed_queries = []
num_queries = 0
for query in queries:
    result_file = Path(f"../{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query, f'experiment query: {experiment[0]} does not match query {query} of current file'

    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            single_table_execution_time = execution_time
        elif "decompose" in algorithm:
            decompose_execution_time = execution_time
        elif "resultdb" in algorithm:
            resultdb_execution_time = execution_time
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"
            
    single_table_decompose.append((single_table_execution_time, decompose_execution_time - single_table_execution_time))
    resultdb.append(resultdb_execution_time)

assert len(single_table_decompose) == len(resultdb) == len(processed_queries), f'number of measurements has to match the processed queries'

# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.30  # the width of the bars
fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.20), layout='constrained')

st_time = [ st_decomp[0] for st_decomp in single_table_decompose ]
decomp_time = [ st_decomp[1] for st_decomp in single_table_decompose ]
ax.bar(x, st_time, width, color=colormap['Single Table'], edgecolor='black', label='Single Table')
ax.bar(x, decomp_time, width, bottom=st_time, color=colormap['Decompose'], edgecolor='black', label='Decompose')

offset = width
ax.bar(x + offset, resultdb, width, color=colormap['Result DB'], edgecolor='black', label='Result DB')

ax.set_ylabel('Query Execution Time [ms]')
# ax.set_title('JOB Algorithm Runtime in mutable')
ax.set_xlabel('JOB Queries')
ax.set_xticks(x + width / 2, processed_queries)
ax.legend(loc='upper right')
ax.set_ylim(0)
ax.xaxis.grid(False) # Disable grid lines on the x-axis
# ax.set_yscale('log')
# fig.legend()
# fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

# legend

# handles, labels = ax.get_legend_handles_labels()
# fig.legend(handles, labels, loc='upper center', ncols=len(methods), bbox_to_anchor=(0.5, 1.25))

fig.savefig('job-algorithm.pdf', bbox_inches='tight')

## Synthetic

### Varying number of joins

In [ ]:
# construct the following dictionary:
# {
#   'Single-Table': [q4_time, q5_time, ...]
#   'Single-Table + Decompose': [q4_time, q5_time, ...]
#   'Result-DB': [q4_time, q5_time, ...]
# }
algorithm_times = {
    'Single Table': [],
    'Single Table + Decompose': [],
    'Result DB': []
}
# ensure that `queries` is sorted!
queries = [
    'star_joins_1',
    'star_joins_2',
    'star_joins_3',
    'star_joins_4'
]
processed_queries = []
num_queries = 0
for query in queries:
    result_file = Path(f"../star-schema/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query or experiment[0] in query, f'experiment query: {experiment[0]} does not match query {query} of current file'
    
    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            algorithm_times['Single Table'].append(execution_time)
        elif "decompose" in algorithm:
            algorithm_times['Single Table + Decompose'].append(execution_time)
        elif "resultdb" in algorithm:
            algorithm_times['Result DB'].append(execution_time)
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"

    assert len(algorithm_times['Single Table']) == len(algorithm_times['Single Table + Decompose']) == len(algorithm_times['Result DB']), f'each algorithm must have the same number of measurements'
            
# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0
fig, ax = plt.subplots(layout='constrained', figsize=(4,3))
for algorithm, time in algorithm_times.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, time, width, label=algorithm)
    # ax.bar_label(rects, padding=3, size=8)
    multiplier += 1

ax.set_ylabel('Runtime [ms]')
ax.set_title('Synthetic Algorithm Runtime in mutable')
ax.set_xticks(x + width, processed_queries)
ax.legend(loc='upper left')
ax.set_ylim(0)
# ax.set_yscale('log')
# fig.legend()
# fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

fig.savefig('synthetic-benchmarks.pdf')

In [ ]:
# ensure that `queries` is sorted!
queries = [
    'star_joins_1',
    'star_joins_2',
    'star_joins_3',
    'star_joins_4'
]
single_table_decompose = []
resultdb = []
processed_queries = []
num_queries = 0

for query in queries:
    result_file = Path(f"../star-schema/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query or experiment[0] in query, f'experiment query: {experiment[0]} does not match query {query} of current file'
    
    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            single_table_execution_time = execution_time
        elif "decompose" in algorithm:
            decompose_execution_time = execution_time
        elif "resultdb" in algorithm:
            resultdb_execution_time = execution_time
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"
            
    single_table_decompose.append((single_table_execution_time, decompose_execution_time - single_table_execution_time))
    resultdb.append(resultdb_execution_time)

assert len(single_table_decompose) == len(resultdb) == len(processed_queries), f'number of measurements has to match the processed queries'

# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0
fig, ax = plt.subplots(layout='constrained', figsize=(4,3))

st_time = [ st_decomp[0] for st_decomp in single_table_decompose ]
decomp_time = [ st_decomp[1] for st_decomp in single_table_decompose ]
ax.bar(x, st_time, width, color=colormap['Single Table'], edgecolor='black', label='Single Table')
ax.bar(x, decomp_time, width, bottom=st_time, color=colormap['Decompose'], edgecolor='black', label='Decompose')

offset = width
ax.bar(x + offset, resultdb, width, color=colormap['Result DB'], edgecolor='black', label='Result DB')

ax.set_ylabel('Runtime [ms]')
ax.set_title('Synthetic Algorithm Runtime in mutable')
ax.set_xticks(x + width / 2, processed_queries)
ax.legend(loc='upper left')
ax.set_ylim(0)
# ax.set_yscale('log')
# fig.legend()
# fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

fig.savefig('synthetic-benchmarks.pdf')

### Varying projections

In [ ]:
# ensure that `queries` is sorted!
queries = [
    'star_proj_2_dim',
    'star_proj_3_dim',
    'star_proj_4_dim',
    'star_proj_fact_1_dim',
    'star_proj_fact_2_dim',
    'star_proj_fact_3_dim',
    'star_proj_fact_4_dim',
]
single_table_decompose = []
resultdb = []
processed_queries = []
num_queries = 0

for query in queries:
    result_file = Path(f"../star-schema/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query or experiment[0] in query, f'experiment query: {experiment[0]} does not match query {query} of current file'
    
    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            single_table_execution_time = execution_time
        elif "decompose" in algorithm:
            decompose_execution_time = execution_time
        elif "resultdb" in algorithm:
            resultdb_execution_time = execution_time
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"
            
    single_table_decompose.append((single_table_execution_time, decompose_execution_time - single_table_execution_time))
    resultdb.append(resultdb_execution_time)

assert len(single_table_decompose) == len(resultdb) == len(processed_queries), f'number of measurements has to match the processed queries'

# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0
fig, ax = plt.subplots(layout='constrained', figsize=(4,3))

st_time = [ st_decomp[0] for st_decomp in single_table_decompose ]
decomp_time = [ st_decomp[1] for st_decomp in single_table_decompose ]
ax.bar(x, st_time, width, color=colormap['Single Table'], edgecolor='black', label='Single Table')
ax.bar(x, decomp_time, width, bottom=st_time, color=colormap['Decompose'], edgecolor='black', label='Decompose')

offset = width
ax.bar(x + offset, resultdb, width, color=colormap['Result DB'], edgecolor='black', label='Result DB')

ax.set_ylabel('Runtime [ms]')
ax.set_title('Synthetic Algorithm Runtime in mutable')
ax.set_xticks(x + width / 2, processed_queries, rotation=45)
ax.legend(loc='upper left')
ax.set_ylim(0)
# ax.set_yscale('log')
# fig.legend()
# fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

fig.savefig('synthetic-benchmarks.pdf')

### Varying selectivity

In [ ]:
# ensure that `queries` is sorted!
queries = [
    'star_sel_10',
    'star_sel_20',
    'star_sel_30',
    'star_sel_40',
    'star_sel_50',
    'star_sel_60',
    'star_sel_70',
    'star_sel_80',
    'star_sel_90',
    'star_sel_100',
]
single_table_decompose = []
resultdb = []
processed_queries = []
num_queries = 0

for query in queries:
    result_file = Path(f"../star-schema/{query}_results.csv")
    if not result_file.is_file():
        print(f"{result_file} does not exist. Skipping...")
        continue
    processed_queries.append(query)
    data = pd.read_csv(result_file)  
    data = (data.groupby(by=['commit', 'date', 'version', 'suite', 'benchmark', 'experiment', 'name', 'config', 'case',], as_index=False)
                .median()
                .drop(['commit', 'date', 'version', 'suite', 'benchmark', 'case', 'runid'], axis=1)
           )
    experiment = data['experiment'].unique()
    assert len(experiment) == 1, f'experiment contains multiple different queries'
    assert experiment[0] == query or experiment[0] in query, f'experiment query: {experiment[0]} does not match query {query} of current file'
    
    for _, row in data.iterrows():
        algorithm = row['name']
        execution_time = row['time']
        if "single-table" in algorithm:
            single_table_execution_time = execution_time
        elif "decompose" in algorithm:
            decompose_execution_time = execution_time
        elif "resultdb" in algorithm:
            resultdb_execution_time = execution_time
        else:
            assert False, "experiment name: {name} does not match any of our algorithms"
            
    single_table_decompose.append((single_table_execution_time, decompose_execution_time - single_table_execution_time))
    resultdb.append(resultdb_execution_time)

assert len(single_table_decompose) == len(resultdb) == len(processed_queries), f'number of measurements has to match the processed queries'

# plot data
x = np.arange(len(processed_queries))  # the label locations
width = 0.30  # the width of the bars
multiplier = 0
fig, ax = plt.subplots(figsize=set_size(fraction_width=1, fraction_height=0.20), layout='constrained')

st_time = [ st_decomp[0] for st_decomp in single_table_decompose ]
decomp_time = [ st_decomp[1] for st_decomp in single_table_decompose ]
ax.bar(x, st_time, width, color=colormap['Single Table'], edgecolor='black', label='Single Table')
ax.bar(x, decomp_time, width, bottom=st_time, color=colormap['Decompose'], edgecolor='black', label='Decompose')

offset = width
ax.bar(x + offset, resultdb, width, color=colormap['Result DB'], edgecolor='black', label='Result DB')

ax.set_ylabel('Query Execution Time [ms]')
# ax.set_title('Synthetic Algorithm Runtime in mutable')
selectivities = [ int(q.rsplit('_', 1)[1]) / 100 for q in processed_queries ]
ax.set_xticks(x + width / 2, selectivities)
ax.set_xlabel('Selectivity')
ax.legend(loc='upper left')
ax.set_ylim(0)
ax.xaxis.grid(False) # Disable grid lines on the x-axis
# ax.set_yscale('log')
# fig.legend()
# fig.legend(ncols=int(len(method_times)/1.5), bbox_to_anchor=(0.5, 1), loc='lower center')

fig.savefig('synthetic-algorithm.pdf', bbox_inches='tight')